In [2]:
import os
import glob
import cv2
from PIL import Image
import numpy as np

# Define the root directory
root_dir = "data"

# Initialize lists for storing data
png_images = {}     
tiff_images = {}
cneg_files = {}
cpos_files = {}
pcd_files = {}

# Loop through folders 01 to 10
for i in range(1, 11):
    folder = os.path.join(root_dir, f"{i:02d}")  # Format 01, 02, ..., 10

    # Load TIFF and PNG images
    rgb_image_files = glob.glob(os.path.join(folder, "*.png"))
    rgb_image_files.sort()
    tiff_image_files = glob.glob(os.path.join(folder, "*.tiff"))
    tiff_image_files.sort()
    for rgb_img_path, tiff_img_path in zip(rgb_image_files, tiff_image_files):
        try:
            png_images[os.path.basename(rgb_img_path)] = Image.open(rgb_img_path)  # Using PIL
            tiff_images[os.path.basename(tiff_img_path)] = Image.open(tiff_img_path)  # Using PIL
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

    # Load TXT files
    all_txt_files = glob.glob(os.path.join(folder, "*.txt"))
    all_txt_files.sort()
    for txt_path in all_txt_files:
        file_name = os.path.basename(txt_path)
        with open(txt_path, "r") as file:
            if "cneg" in file_name:
                cneg_files[file_name] = np.loadtxt(file)
            elif "cpos" in file_name:
                cpos_files[file_name] = np.loadtxt(file)
            else:
                pcd_files[file_name] = txt_path


/tmp/ipykernel_37685/1998358913.py:40: UserWarning: loadtxt: input contained no data: "<_io.TextIOWrapper name='data/01/pcd0154cneg.txt' mode='r' encoding='UTF-8'>"
  cneg_files[file_name] = np.loadtxt(file)


In [ ]:
# Print sample outputs
print(f"Total rgb images loaded: {len(png_images)}")
print(f"Total tiff images loaded: {len(tiff_images)}")
print(f"Total cneg_files loaded: {len(cneg_files)}")
print(f"Total cpos_files loaded: {len(cpos_files)}")

In [3]:
cnegs = list(cneg_files.values())
cposs = list(cpos_files.values())
png_images_list = list(png_images.values())

In [4]:
png_images_names = list(png_images.keys())

In [ ]:
for i, cneg in enumerate(cnegs):
    image_overlay = np.array(png_images[i])
    for j in range(int(len(cneg)/4)):
        rect = cneg[j*4:(j+1)*4]
        pts = np.array(rect, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image_overlay, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

for i, cpos in enumerate(cposs):
    # image_overlay = np.array(png_images[i])
    for j in range(int(len(cpos)/4)):
        rect = cpos[j*4:(j+1)*4]
        pts = np.array(rect, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image_overlay, [pts], isClosed=True, color=(0, 255, 0), thickness=2)


In [ ]:
import os
import cv2
import numpy as np


output_folder = 'output_images/with_bboxes'
os.makedirs(output_folder, exist_ok=True)

for i, (cneg, cpos) in enumerate(zip(cnegs, cposs)):
    image_overlay = np.array(png_images_list[i])
    
    # Process cneg (negative bounding boxes)
    for j in range(int(len(cneg)/4)):
        rect = cneg[j*4:(j+1)*4]
        pts = np.array(rect, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image_overlay, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

    # Process cpos (positive bounding boxes)
    for j in range(int(len(cpos)/4)):
        rect = cpos[j*4:(j+1)*4]
        pts = np.array(rect, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image_overlay, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Save the image to the output folder
    output_path = os.path.join(output_folder, png_images_names[i])
    cv2.imwrite(output_path, image_overlay)


In [ ]:
import argparse
import glob
import os
import numpy as np
from imageio import imsave
from depth_image import DepthImage

pcds = list(pcd_files.values())

for pcd in pcds:
    di = DepthImage.from_pcd(pcd, (480, 640))
    di.inpaint()

    of_name = pcd.replace('.txt', 'd.tiff')
    imsave(of_name, di.img.astype(np.float32))

In [22]:
# Loop through folders 01 to 10
tiff_images = {}
for i in range(1, 11):
    folder = os.path.join(root_dir, f"{i:02d}")  # Format 01, 02, ..., 10
    tiff_image_files = glob.glob(os.path.join(folder, "*.tiff"))
    tiff_image_files.sort()
    for tiff_img_path in tiff_image_files:
        try:
            tiff_images[os.path.basename(tiff_img_path)] = Image.open(tiff_img_path)  # Using PIL
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

In [5]:
tiff_images_list = list(tiff_images.values())

In [6]:
tiff_images_names = list(tiff_images.keys())

In [41]:
output_folder = "output_images/rgbd/"
os.makedirs(output_folder, exist_ok=True)
for rgb_img, depth_img, name in zip(png_images_list, tiff_images_list, tiff_images_names):
    # Convert RGB image to a NumPy array
    rgb_array = np.array(rgb_img)
    
    # Convert depth image to NumPy array (float32)
    depth_array = np.array(depth_img, dtype=np.float32)
    
    depth_array = cv2.normalize(depth_array, None, 0, 255, cv2.NORM_MINMAX)
    depth_array = depth_array.astype(np.uint8)
    depth_array = np.expand_dims(depth_array, axis=-1)
    
    # Repeat depth image across 3 channels to make it compatible with RGB
    # depth_array_3d = np.repeat(depth_array[:, :, np.newaxis], 3, axis=2)
    
    # Stack RGB image and depth image to form RGB-D image (4 channels)
    rgbd_image = np.concatenate((rgb_array, depth_array), axis=2)
    # Convert back to PIL Image (for saving)
    rgbd_image_pil = Image.fromarray(rgbd_image)
    rgbd_image_pil.save(output_folder+name.replace("d.tiff", "rgbd.png"))